# Economic Loss Calculation

In [17]:
import pandas as pd

In [49]:
def get_state_name(lat, long):
    from geopy.geocoders import Nominatim
    locator = Nominatim(user_agent="myGeocoder")
    coordinates = str(lat)+", "+str(long)
    location = locator.reverse(coordinates)
    return location.raw['address']["state"]

In [72]:
def get_value_per_acre(lat, long):
    propertydata = pd.read_csv("Data/statewise-pricing-usa.csv")
    state = get_state_name(lat, long)
    return propertydata.loc[propertydata['State'] == state].values[0,1]

In [113]:
def get_total_economic_loss(lat, long, area):
    acres = 247.105 * area
    return get_value_per_acre(lat, long) * acres

def get_area_from_scan_track(scan, track):
    l = scan * 111 #km
    h = track * 111 #km
    area_in_km2 = l*h
    return area_in_km2

def get_weighted_economic_loss(total_economic_loss, risk, risk_scale): #risk_scale is maximum value of risk possible (such as out of 5 or out of 100 etc)
    if(risk > risk_scale):
        raise ArithmeticError("Risk cannot be greater than risk scale")
    return total_economic_loss * risk / risk_scale

In [111]:
# testing code
totalloss = get_total_economic_loss(32.9715645, -89.7348497, get_area_from_scan_track(3.5,1.8)) # roughly 0.00404686 km2 = 1 acre
weightedloss = get_weighted_economic_loss(totalloss, 3.7, 5)
print(totalloss, weightedloss)

1382939893632.1501 1023375521287.7913


# Location Name to Coordinates Tuple

In [70]:
def get_lat_long_from_location(name):
    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent="myGeocoder")
    location = geolocator.geocode(name)
    return (location.latitude,location.longitude)

In [94]:
get_lat_long_from_location('Mississipi')

(32.9715645, -89.7348497)


# Ecological Loss Calculation

### Endangered Species

In [13]:
def get_county_name(lat, long):
    from geopy.geocoders import Nominatim
    locator = Nominatim(user_agent="myGeocoder")
    coordinates = str(lat)+", "+str(long)
    location = locator.reverse(coordinates)
    county = location.raw['address']["county"]
    if 'County' in county:
        county = county.replace(' County','')
    return county

In [36]:
def get_df_endangered_species(lat, long):
    county = get_county_name(lat, long)
    wild = pd.read_csv("Data/wildlife.csv")
    wild = wild.loc[(wild['County'] == county) & (wild['Global Conservation Rank'].isin(['G1','G2']))]['Common Name']
    if wild.empty:
        return None
    return wild

In [38]:
get_df_endangered_species(42.6526, -73.7562) # if none return or no output - No endangered species